<a href="https://colab.research.google.com/github/RajT393/rexode-server-auto/blob/main/cloud_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!ngrok config add-authtoken 2z7jsSLPPfldAoVDwYZcOYTLZ29_7RodgXQkxLNfHHUCrR6Uv

# ✅ Install dependencies
!pip install -q transformers accelerate fastapi uvicorn pyngrok

# ✅ Import Libraries
from transformers import AutoTokenizer, AutoModelForCausalLM
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import uvicorn
import torch
import nest_asyncio
import threading
from pyngrok import ngrok
import requests

# ✅ Model & Tokenizer
model_id = "deepseek-ai/deepseek-coder-6.7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

# ✅ FastAPI Setup
app = FastAPI()
app.add_middleware(CORSMiddleware, allow_origins=["*"], allow_methods=["*"], allow_headers=["*"])

class PromptRequest(BaseModel):
    prompt: str

@app.post("/generate")
async def generate(req: PromptRequest):
    inputs = tokenizer(req.prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.7)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return {"response": decoded}

# ✅ Run Server
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

!ngrok config add-authtoken 2z7jsSLPPfldAoVDwYZcOYTLZ29_7RodgXQkxLNfHHUCrR6Uv
# ✅ Start tunnel & server
def start():
    public_url = ngrok.connect(8000)
    print("🚀 Public URL:", public_url)

    # OPTIONAL: Upload to Supabase (replace with your project + key)
    # requests.patch("https://xyz.supabase.co/rest/v1/config?name=eq.llm_url", headers={...}, json={"value": str(public_url)})

    thread = threading.Thread(target=run_server)
    thread.start()

nest_asyncio.apply()
start()


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🚀 Public URL: NgrokTunnel: "https://4dfb-34-87-58-146.ngrok-free.app" -> "http://localhost:8000"
